In [1]:
!pip install backtrader
!pip install yfinance

In [2]:
from google.colab import drive
drive.mount("/content/drive")

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/Quant/Backtrader

In [ ]:
import backtrader as bt 
import datetime
import yfinance as yf
from matplotlib import pyplot as plt

# ***RSI Strategy***

In [ ]:
class RSIStrategy(bt.Strategy):

  params = (
    ('maperiod', 15),
    ) #Window size for RSI
  def log(self, text): #Function to print out log details (Closing price & Date)
    dt = self.datas[0].datetime.date(0)
    print(dt.isoformat(), text) 
    
  def __init__(self):
    self.dataclose = self.datas[0].close
    self.order = None
    self.rsi = bt.indicators.RSI(self.datas[0]) #Initialise the inidicator RSI

  def notify_order(self, order): #Notify the user when a new order is executed/cancelled
    if order.status in [order.Submitted, order.Accepted]: #If order status is accepted/submitted, no need to notify as it is not yet executed.
      return
        
    if order.status in [order.Completed]: #If order is executed then notify the user
      if order.isbuy(): #If buy order
        self.log('BUY EXECUTED :Price- %2f, Cost- %2f, Commission- %2f' % (order.executed.price, order.executed.value, order.executed.comm))
      elif order.issell(): #If sell order
        self.log('SELL EXECUTED :Price- %2f, Cost- %2f, Commission- %2f' % (order.executed.price, order.executed.value, order.executed.comm))
      self.bar_executed = len(self)

    elif order.status in [order.Canceled, order.Margin, order.Rejected]: #If order is cancelled, notify the same
      self.log('ORDER CANCELLED/ MARGIN/ REJECTED')
    self.order = None


  def notify_trade(self, trade): #Notify the user about profits after squarring off a trade
    if not trade.isclosed: 
      return
    self.log('OPERATION PROFIT: GROSS- %2f, NET- %2f' % (trade.pnl, trade.pnlcomm))
    
  def next(self):
    if self.order: #if order is pending, cant send another order, go ahead to next day
      return

    #if we are not in market position, we might buyp
    if not self.position:
      if self.rsi[0] <30:
        self.log('BUY CREATE - %2f' % self.dataclose[0])
        self.order = self.buy() #place a buy order, notify_order is executed for buy
      if self.rsi > 70:
        self.log('SELL CREATE - %2f' % self.dataclose[0])
        self.order = self.buy() #place a buy order, notify_order is executed for buy
        
    else: #we might sell
      if self.order == None and self.broker.get_value() > self.dataclose and self.rsi <30:
        #this block is to accommodate muptiple positions at the same time irrespective of our market position
        self.log('BUY CREATE - %2f' % self.dataclose[0])
        self.order = self.buy() #place a buy order, notify_order is executed for buy 
      if self.rsi[0] > 70:
        self.log('SELL CREATE - %2f' % self.dataclose[0])
        self.order = self.sell() #place a sell order, notify_order is executed for sell

In [ ]:
cerebro_rsi = bt.Cerebro() #Initialise the bot

#add the strategy
cerebro_rsi.addstrategy(RSIStrategy)
#sets the sub-folder from where data feed will be collected
datapath = 'AAPL.csv'
#set the data feed
data = bt.feeds.PandasData(dataname=yf.download('AAPL', '2021-01-01', '2021-04-01'))

cerebro_rsi.adddata(data) #add data feed to engine

cerebro_rsi.broker.setcash(50000.0) #Set initial portfolio size
cerebro_rsi.broker.setcommission(commission=0.001) #Broker commission set as 0.1%
cerebro_rsi.addsizer(bt.sizers.FixedSize, stake = 10) #No of shares transacted during an order, can be manually specified in the buy() or sell() function
initial = cerebro_rsi.broker.get_value()
print('INITIAL PORTFOLIO VALUE: ',initial) #print initial portfolio value
cerebro_rsi.run()
final = cerebro_rsi.broker.get_value()
print('\n\nFINAL PORTFOLIO VALUE: ',final) #print final portfolio value
print('PROFIT: ', (final - initial)*100/initial, '%') #print profit
figure = plt.gcf()
figure.set_size_inches(10,5)
cerebro_rsi.plot()[0][0].savefig('figure_RSI.png', dpi=300) #save figure as image file

# ***MULTIPLE INDICATOR STRATEGY***
Buy when:

1. Closes above the top line of Bolinger Band (period = 20)
2. RSI > 50
3. Volume > SMA(Volume, period = 12)

Sell when:
1. Closes below bottom line of Bollinger Band
2. RSI < 50

In [ ]:
class MultipleStrategy(bt.Strategy):

  params = (('BBandsperiod', 20), ('VolumePeriod', 12),)
  def log(self, text):
    dt = self.datas[0].datetime.date(0)
    print(dt.isoformat(), text)
    
  def __init__(self):
    self.dataclose = self.datas[0].close
    self.order = None
    self.rsi = bt.indicators.RSI(self.datas[0])
    self.bband = bt.indicators.BBands(self.datas[0], period=self.params.BBandsperiod)
    self.volume_sma = bt.indicators.SMA(self.datas[0].volume, period = self.params.VolumePeriod)
    self.topline = None
    self.botline = None


  def notify_order(self, order):
    if order.status in [order.Submitted, order.Accepted]:
      return
        
    if order.status in [order.Completed]:
      if order.isbuy():
        self.log('BUY EXECUTED :Price- %2f, Cost- %2f, Commission- %2f' % (order.executed.price, order.executed.value, order.executed.comm))
      elif order.issell():
        self.log('SELL EXECUTED :Price- %2f, Cost- %2f, Commission- %2f' % (order.executed.price, order.executed.value, order.executed.comm))
      self.bar_executed = len(self)

    elif order.status in [order.Canceled, order.Margin, order.Rejected]:
      self.log('ORDER CANCELLED/ MARGIN/ REJECTED')
    self.order = None


  def notify_trade(self, trade):
    if not trade.isclosed:
      return
    self.log('OPERATION PROFIT: GROSS- %2f, NET- %2f' % (trade.pnl, trade.pnlcomm))
  
    
  def next(self):
    if self.order: #if order is pending, cant send another order, go ahead to next day
      return

    if self.dataclose[0] > self.bband.lines.top:
      self.topline = True
    if self.dataclose[0] < self.bband.lines.bot:
      self.botline = True

    #if we are not in market, we might buyp
    if not self.position:
      if self.topline and self.rsi > 50 and self.volume_sma > self.datas[0].volume:
        self.log('BUY CREATE - %2f' % self.dataclose[0])
        self.order = self.buy() #notify_order is executed for buy
      if self.botline and self.rsi <50:
        self.log('SELL CREATE - %2f' % self.dataclose[0])
        self.order = self.buy()
        
    else: #we might sell
      if self.order == None and self.broker.get_value() > self.dataclose and self.topline and self.rsi > 50 and self.volume_sma > self.datas[0].volume:
        self.log('BUY CREATE - %2f' % self.dataclose[0])
        self.order = self.buy()
      if self.botline and self.rsi <50:
        self.log('SELL CREATE - %2f' % self.dataclose[0])
        self.order = self.sell()

In [ ]:
cerebro_multi = bt.Cerebro()

#add the strategy
cerebro_multi.addstrategy(MultipleStrategy)
#sets the sub-folder from where data feed will be collected
datapath = 'AAPL.csv'
#set the data feed
data = bt.feeds.PandasData(dataname=yf.download('AAPL', '2021-01-01', '2021-04-01'))

cerebro_multi.adddata(data) #add to engine

cerebro_multi.broker.setcash(50000.0)
cerebro_multi.broker.setcommission(commission=0.001) #0.1%
cerebro_multi.addsizer(bt.sizers.FixedSize, stake = 10)
initial = cerebro_multi.broker.get_value()
print('INITIAL PORTFOLIO VALUE: ',initial)
cerebro_multi.run()
final = cerebro_multi.broker.get_value()
print('\n\nFINAL PORTFOLIO VALUE: ',final)
print('PROFIT: ', (final - initial)*100/initial, '%')
figure = plt.gcf()
# figure.set_size_inches(20,20)
# cerebro_multi.plot()[0][0].savefig('figure_MultipleStrategy.png', dpi=100)

In [ ]:
class GoldenCross(bt.Strategy):

  params = (('fast', 50), ('slow', 100),('ticker', 'AAPL'),('order_percentage',0.95))
  def log(self, text): #Function to print out log details (Closing price & Date)
    dt = self.datas[0].datetime.date(0)
    print(dt.isoformat(), text) 

  def __init__(self):
    self.dataclose = self.datas[0].close
    self.order = None
    self.fast_moving_average=bt.indicators.SMA(
        self.data.close,period=self.params.fast,plotname='50 day moving average'
    )
    self.slow_moving_average=bt.indicators.SMA(
        self.data.close,period=self.params.slow,plotname='200 day moving average'
    )
    self.crossover=bt.indicators.CrossOver(self.fast_moving_average,self.slow_moving_average)

  def notify_order(self, order): #Notify the user when a new order is executed/cancelled
    if order.status in [order.Submitted, order.Accepted]: #If order status is accepted/submitted, no need to notify as it is not yet executed.
      return
        
    if order.status in [order.Completed]: #If order is executed then notify the user
      if order.isbuy(): #If buy order
        self.log('BUY EXECUTED :Price- %2f, Cost- %2f, Commission- %2f' % (order.executed.price, order.executed.value, order.executed.comm))
      elif order.issell(): #If sell order
        self.log('SELL EXECUTED :Price- %2f, Cost- %2f, Commission- %2f' % (order.executed.price, order.executed.value, order.executed.comm))
      self.bar_executed = len(self)

    elif order.status in [order.Canceled, order.Margin, order.Rejected]: #If order is cancelled, notify the same
      self.log('ORDER CANCELLED/ MARGIN/ REJECTED')
    self.order = None

  def next(self):
    if self.order: #if order is pending, cant send another order, go ahead to next day
      return

    #if we are not in market position, we might buyp
    if not self.position:
      if self.crossover >0:
        self.log('BUY CREATE - %2f' % self.dataclose[0])
        self.order = self.buy() #place a buy order, notify_order is executed for buy
      if self.crossover < 0:
        self.log('SELL CREATE - %2f' % self.dataclose[0])
        self.order = self.buy() #place a buy order, notify_order is executed for buy
        
    else: #we might sell
      if self.order == None and self.broker.get_value() > self.dataclose and self.crossover >0:
        #this block is to accommodate muptiple positions at the same time irrespective of our market position
        self.log('BUY CREATE - %2f' % self.dataclose[0])
        self.order = self.buy() #place a buy order, notify_order is executed for buy 
      if self.crossover < 0:
        self.log('SELL CREATE - %2f' % self.dataclose[0])
        self.order = self.sell() #place a sell order, notify_order is executed for sell

In [ ]:
cerebro_rsi = bt.Cerebro() #Initialise the bot

#add the strategy
cerebro_rsi.addstrategy(GoldenCross)
#sets the sub-folder from where data feed will be collected
datapath = 'AAPL.csv'
#set the data feed
data = bt.feeds.PandasData(dataname=yf.download('AAPL', '2021-01-01', '2021-04-01'))

cerebro_rsi.adddata(data) #add data feed to engine

cerebro_rsi.broker.setcash(50000.0) #Set initial portfolio size
cerebro_rsi.broker.setcommission(commission=0.001) #Broker commission set as 0.1%
cerebro_rsi.addsizer(bt.sizers.FixedSize, stake = 10) #No of shares transacted during an order, can be manually specified in the buy() or sell() function
initial = cerebro_rsi.broker.get_value()
print('INITIAL PORTFOLIO VALUE: ',initial) #print initial portfolio value
cerebro_rsi.run()
final = cerebro_rsi.broker.get_value()
print('\n\nFINAL PORTFOLIO VALUE: ',final) #print final portfolio value
print('PROFIT: ', (final - initial)*100/initial, '%') #print profit
figure = plt.gcf()
figure.set_size_inches(10,5)
cerebro_rsi.plot()[0][0].savefig('figure_RSI.png', dpi=300) #save figure as image file

In [ ]:
class MACD(bt.Strategy):

  params = (
        # Standard MACD Parameters
        ('macd1', 12),
        ('macd2', 26),
        ('macdsig', 9),
        ('atrperiod', 14),  # ATR Period (standard)
        ('atrdist', 3.0),   # ATR distance for stop price
        ('smaperiod', 30),  # SMA Period (pretty standard)
        ('dirperiod', 10),  # Lookback period to consider SMA trend direction
    )
  def log(self, text): #Function to print out log details (Closing price & Date)
    dt = self.datas[0].datetime.date(0)
    print(dt.isoformat(), text) 

  def __init__(self):
    self.dataclose = self.datas[0].close
    self.order = None
    self.macd = bt.indicators.MACD(self.data,
                                       period_me1=self.p.macd1,
                                       period_me2=self.p.macd2,
                                       period_signal=self.p.macdsig)

        # Cross of macd.macd and macd.signal
    self.mcross = bt.indicators.CrossOver(self.macd.macd, self.macd.signal)

        # To set the stop price
    self.atr = bt.indicators.ATR(self.data, period=self.p.atrperiod)

        # Control market trend
    self.sma = bt.indicators.SMA(self.data, period=self.p.smaperiod)
    self.smadir = self.sma - self.sma(-self.p.dirperiod)

  def notify_order(self, order): #Notify the user when a new order is executed/cancelled
    if order.status in [order.Submitted, order.Accepted]: #If order status is accepted/submitted, no need to notify as it is not yet executed.
      return
        
    if order.status in [order.Completed]: #If order is executed then notify the user
      if order.isbuy(): #If buy order
        self.log('BUY EXECUTED :Price- %2f, Cost- %2f, Commission- %2f' % (order.executed.price, order.executed.value, order.executed.comm))
      elif order.issell(): #If sell order
        self.log('SELL EXECUTED :Price- %2f, Cost- %2f, Commission- %2f' % (order.executed.price, order.executed.value, order.executed.comm))
      self.bar_executed = len(self)

    elif order.status in [order.Canceled, order.Margin, order.Rejected]: #If order is cancelled, notify the same
      self.log('ORDER CANCELLED/ MARGIN/ REJECTED')
    self.order = None

  def next(self):
    if self.order: #if order is pending, cant send another order, go ahead to next day
      return

    #if we are not in market position, we might buyp
    if not self.position:
      if self.mcross[0] > 0.0 and self.smadir < 0.0:
        self.log('BUY CREATE - %2f' % self.dataclose[0])
        self.order = self.buy() #place a buy order, notify_order is executed for buy
        pdist = self.atr[0] * self.p.atrdist
        self.pstop = self.data.close[0] - pdist
      if self.mcross[0] < 0.0 and self.smadir > 0.0:
        self.log('SELL CREATE - %2f' % self.dataclose[0])
        self.order = self.buy() #place a buy order, notify_order is executed for buy
        
    else: #we might sell
      if self.order == None and self.broker.get_value() > self.dataclose and self.mcross[0] > 0.0 and self.smadir < 0.0:
        #this block is to accommodate muptiple positions at the same time irrespective of our market position
        self.log('BUY CREATE - %2f' % self.dataclose[0])
        self.order = self.buy() #place a buy order, notify_order is executed for buy 
      if self.mcross[0] < 0.0 and self.smadir > 0.0:
        self.log('SELL CREATE - %2f' % self.dataclose[0])
        self.order = self.sell() #place a sell order, notify_order is executed for sell

In [ ]:
cerebro_rsi = bt.Cerebro() #Initialise the bot

#add the strategy
cerebro_rsi.addstrategy(MACD)
#sets the sub-folder from where data feed will be collected
datapath = 'AAPL.csv'
#set the data feed
data = bt.feeds.PandasData(dataname=yf.download('AAPL', '2021-01-01', '2021-04-01'))

cerebro_rsi.adddata(data) #add data feed to engine

cerebro_rsi.broker.setcash(50000.0) #Set initial portfolio size
cerebro_rsi.broker.setcommission(commission=0.001) #Broker commission set as 0.1%
cerebro_rsi.addsizer(bt.sizers.FixedSize, stake = 10) #No of shares transacted during an order, can be manually specified in the buy() or sell() function
initial = cerebro_rsi.broker.get_value()
print('INITIAL PORTFOLIO VALUE: ',initial) #print initial portfolio value
cerebro_rsi.run()
final = cerebro_rsi.broker.get_value()
print('\n\nFINAL PORTFOLIO VALUE: ',final) #print final portfolio value
print('PROFIT: ', (final - initial)*100/initial, '%') #print profit
figure = plt.gcf()
figure.set_size_inches(10,5)
cerebro_rsi.plot()[0][0].savefig('figure_RSI.png', dpi=300) #save figure as image file